In [ ]:
import pandas as pd
import gurobipy as grb

def importData():
    participants = pd.read_csv("USU Internals Week 2 S1 2019 Signup (Responses) - Form Responses 1.csv")
    
    global names
    global pros
    global novices
    global strengths
    names = []
    pros = []
    novices = []
    strengths = {}

    global vetoesByPerson
    vetoesByPerson = {}

    global prefsByPerson
    prefsByPerson = {}

    proimportindex = 1
    noviceimportindex = 1
    masterindex = 1
    # Check for duplicates
    for ind,row in participants.iterrows():
        name = row[1]
        status = row[2]
        novExp = row[4]
        prefs = row[5]
        vetoes = row[9]

        if name not in names and name != "nan":
            names.append(row[1])
            if status == "Pro":
                pros.append(row[1])
            elif status == "Novice":
                novices.append(row[1])
                strengths[name] = "Strong" if novExp == "Yes" else "Weak"

            prefsByPerson[name] = prefs.split(", ")
            vetoesByPerson[name] = vetoes.split(", ")

def calcCosts(allowPros,allowAllNov):
    print("{} Pros, {} Novices.".format(len(pros),len(novices)))
    if len(pros) < len(novices)/2 and not allowAllNov:
        print("{} Pros, {} Novices - Insufficient pros to proceed with allocation.".format(len(pros),len(novices)))
    else:
        print("{} Pros, {} Novices.".format(len(pros),len(novices)))
        
    global isAllNov
    isAllNov = {}
    
    global fts
    fts = {}
    ftindex = 1
    if allowPros:
        for pro in pros:
            for novice1 in novices:
                for novice2 in novices:
                    if novice1 != novice2:
                        fts[ftindex] = pro,novice1,novice2
                        ftindex += 1
    if allowAllNov:
        for novice1 in novices:
            for novice2 in novices:
                for novice3 in novices:
                    if novice1 != novice2 and novice1 != novice3 and novice3 != novice2:
                        fts[ftindex] = novice1,novice2,novice3
                        ftindex +=1


    print("{} possible team configurations".format(len(fts)))
    global costs
    costs = {}
    
    for key in fts.keys():
        costs[key] = 0

    for key,team in fts.iteritems():
        novRanks = []
        prosInTeam = 0
        for speaker in team:
            try:
                novRanks.append(strengths[speaker])
            except:
                prosInTeam += 1
    
        if prosInTeam == 0:
            costs[key] = 100000 # Base Costs
            strongs = 0
            weaks = 0
            
            for i in (0,1,2):
                if novRanks[i] == "Strong":
                    strongs += 1
                elif novRanks[i] == "Weak":
                    weaks += 1
            
            if strongs == 3:
                costs[key] += 0
                isAllNov[key] = "3 Strong"
            elif strongs == 2:
                costs[key] += 2000
                isAllNov[key] = "2 Strong"
            elif strongs == 1:
                costs[key] += 4000
                isAllNov[key] = "1 Strong"
            elif strongs == 0:
                costs[key] += 8000
                isAllNov[key] = "0 Strong"
                
            # Infinitely high cost for allNov teams with Swings
            if "Swing" in team:
                costs[key] = grb.GRB.INFINITY
            
        else:
            costs[key] = 0 # Base Costs
            
            if novRanks[0] == "Strong" and novRanks[1] == "Strong":
                costs[key] += 0
            elif novRanks[0] == "Strong" and novRanks[1] == "Weak":
                costs[key] += 2000
            elif novRanks[0] == "Weak" and novRanks[1] == "Strong":
                costs[key] += 2000
            else: # 2 weak
                costs[key] += 4000


    # Auto-import vetoes and preferences   
    for name,preferences in prefsByPerson.iteritems():
        for p in preferences:
            preference(name,p)
    
    for name,vetoes in vetoesByPerson.iteritems():
        for v in vetoes:
            veto(name,v)
            
def allocate():
    print("---")
    m = grb.Model("Allocate")

    x = {}

    for key,value in costs.iteritems():
        x[key] = m.addVar(vtype = grb.GRB.BINARY, obj = value, name="x_{}".format(key))

    # Each person must be on exactly one team

    for person in pros:
        affectedTeams = []
        for teamkey,team in fts.iteritems():
            if person in team:
                affectedTeams.append(teamkey)
        m.addConstr(sum(x[affectedTeam] for affectedTeam in affectedTeams) == 1)

    for person in novices:
        affectedTeams = []
        for teamkey,team in fts.iteritems():
            if person in team:
                affectedTeams.append(teamkey)
        m.addConstr(sum(x[affectedTeam] for affectedTeam in affectedTeams) == 1)

    m.modelSense = grb.GRB.MINIMIZE

    m.optimize()
    count = 0
    for key,value in costs.iteritems():
        if x[key].x != 0:
            try:
                novstring = isAllNov[key]
            except:
                novstring = ""
            
            if key in isAllNov.keys():
                comment = "All Novice - {}".format(isAllNov[key])
            else:
                comment = "Pro-Am"
                
            print("{} (Cost {}) ({})".format(fts[key],value,comment))
            count += 1
    print("---")       
    print("{} teams".format(count))
    
def preference(p1,p2):
    if p1 != "--" and p2 != "--":
        for key,value in costs.iteritems():
            t = fts[key]
            if p1 in t and p2 in t:
                costs[key] -= 10000
        print("{0} preferences {1}".format(p1,p2))

def veto(p1,p2):
    if p1 != "--" and p2 != "--":
        for key,value in costs.iteritems():
            t = fts[key]
            if p1 in t and p2 in t:
                costs[key] = grb.GRB.INFINITY
        print("{0} vetoes {1}".format(p1,p2))

In [ ]:
importData()
calcCosts(True,True)
allocate()

In [ ]:
"""
    dummy12 = []
    for key,team in fts.iteritems():
        if "Dummy1" in team and "Dummy2" in team:
            dummy12.append(key)
    m.addConstr(sum(x[team] for team in dummy12) == 1)
"""